This Notebook trains models on the feature engineered data.



In [1]:
# set path for imports  # TODO: move notebooks folder to root level and install windfarm_forecast package in dev mode with poetry to avoid adjust the path in each notebook
import sys
import os
sys.path.append(os.path.abspath('../..'))

In [2]:
from windfarm_forecast.utils import setup_mlflow

# Set up MLflow experiment
experiment_id = setup_mlflow()

Traceback (most recent call last):
  File "/Users/fischere/Library/CloudStorage/OneDrive-SchwarzIT/05 Learning/pprojects/windpark_forecasting_kdd22/windfarm_forecast/.venv/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 328, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/Users/fischere/Library/CloudStorage/OneDrive-SchwarzIT/05 Learning/pprojects/windpark_forecasting_kdd22/windfarm_forecast/.venv/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 422, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/Users/fischere/Library/CloudStorage/OneDrive-SchwarzIT/05 Learning/pprojects/windpark_forecasting_kdd22/windfarm_forecast/.venv/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1368, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/Users/fischere/Library/CloudStorage/OneDrive-SchwarzIT/05 Le

Active experiment: windfarm_power_prediction


## Load data

In [3]:
import pandas as pd

# load train and validation data
file_path = '../../data/modified/feature_engineered/'
df_train = pd.read_parquet(file_path + 'sdwpf_train_214days_v1_feature_engineered.parquet')
df_val = pd.read_parquet(file_path + 'sdwpf_validation_31days_v1_feature_engineered.parquet')

In [4]:
df_train

,timestamp,num_turbines,Day,time,Wspd,Wdir,Etmp,Itmp,Ndir,maintenance_day,turbine_stopped,turbine_at_rest,impute_day_patv,Patv,Patv_imputed,n_active_turbines
0,2020-05-01 00:00:00,134,1.0,00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.000000,0.007463,0.00,0.00000,0
1,2020-05-01 00:10:00,134,1.0,00:10,5.653284,-5.581119,31.184240,36.925522,20.347015,0.0,0.022388,0.029851,0.007463,46766.08,47.15532,131
2,2020-05-01 00:20:00,134,1.0,00:20,5.514104,-5.615970,31.110720,36.851493,19.238955,0.0,0.022388,0.029851,0.007463,44853.70,45.21796,131
3,2020-05-01 00:30:00,134,1.0,00:30,5.641053,-5.070977,31.055403,36.803158,19.474211,0.0,0.022388,0.029851,0.007463,46268.76,46.67680,130
4,2020-05-01 00:40:00,134,1.0,00:40,5.360075,-5.564436,30.956129,36.659474,17.313233,0.0,0.022388,0.029851,0.007463,41189.26,41.54024,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30811,2020-11-30 23:10:00,134,214.0,23:10,4.860224,1.989701,1.449764,11.251866,252.094701,0.0,0.000000,0.000000,0.000000,43688.69,43.68869,134
30812,2020-11-30 23:20:00,134,214.0,23:20,4.600299,2.525448,1.405433,11.197388,256.603731,0.0,0.000000,0.000000,0.000000,40150.28,40.15028,134
30813,2020-11-30 23:30:00,134,214.0,23:30,4.288134,2.386493,1.315039,11.132687,259.334030,0.0,0.000000,0.000000,0.000000,35560.85,35.56085,134
30814,2020-11-30 23:40:00,134,214.0,23:40,4.205299,1.267388,1.269055,11.099701,261.968209,0.0,0.000000,0.000000,0.000000,34561.02,34.56102,134


In [5]:
df_val

,timestamp,num_turbines,Day,time,Wspd,Wdir,Etmp,Itmp,Ndir,maintenance_day,turbine_stopped,turbine_at_rest,impute_day_patv,Patv,Patv_imputed,n_active_turbines
0,2020-12-01 00:00:00,134,215.0,00:00,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,0.0,0.0,0.00,0.00000,0
1,2020-12-01 00:10:00,134,215.0,00:10,4.733358,0.610821,1.112992,11.007313,261.712388,0.0,0.007463,0.0,0.0,42232.92,42.23292,133
2,2020-12-01 00:20:00,134,215.0,00:20,4.677612,1.387761,1.063937,10.991791,264.046418,0.0,0.000000,0.0,0.0,45467.10,45.46710,134
3,2020-12-01 00:30:00,134,215.0,00:30,5.017612,-1.300373,0.978110,10.967015,264.594701,0.0,0.000000,0.0,0.0,51994.43,51.99443,134
4,2020-12-01 00:40:00,134,215.0,00:40,5.503134,-1.750075,0.879764,10.921194,263.415746,0.0,0.000000,0.0,0.0,62983.80,62.98380,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,2020-12-31 23:10:00,134,245.0,23:10,5.888582,-0.473657,-0.445118,10.075149,196.448881,0.0,0.000000,0.0,0.0,60320.66,60.32066,134
4460,2020-12-31 23:20:00,134,245.0,23:20,6.023759,-1.024211,-0.501508,10.106391,196.958496,0.0,0.000000,0.0,0.0,68652.84,68.65284,133
4461,2020-12-31 23:30:00,134,245.0,23:30,6.110373,-1.207687,-0.552126,10.084925,195.546269,0.0,0.000000,0.0,0.0,78947.44,78.94744,134
4462,2020-12-31 23:40:00,134,245.0,23:40,6.025448,-2.057015,-0.621024,10.080448,194.121940,0.0,0.000000,0.0,0.0,75742.24,75.74224,134


In [6]:
# set timestamp as index
df_train = df_train.set_index('timestamp', drop=True)
df_val = df_val.set_index('timestamp', drop=True)

Idea: compare the difference approaches to such a timeseries prediction problem: direct single step, multi-step direct, recursive, multi-output, seq2seq (e.g. GRU or LSTM)

In [7]:
# define feature and target columns
features = ['Wspd', 'Wdir', 'Etmp', 'n_active_turbines']
target = 'Patv_imputed'

## Helper Functions

In [8]:
import plotly.express as px
import pandas as pd
import numpy as np

def plot_predictions(y_true: pd.Series, y_pred: np.ndarray | list, title: str = 'Predictions vs Actual Values') -> None:
    """
    Creates an interactive line plot comparing predicted vs actual wind farm power output values.
    
    Args:
        y_true (pd.Series): True power output values with datetime index
        y_pred (array-like): Predicted power output values corresponding to y_true
        title (str, optional): Title for the plot. Defaults to 'Predictions vs Actual Values'
    
    Returns:
        None.
    """
    plot_df = pd.concat([
        pd.DataFrame({'Time': y_true.index, 'Power Output (kW)': y_true.values, 'Type': 'Actual'}),
        pd.DataFrame({'Time': y_true.index, 'Power Output (kW)': y_pred, 'Type': 'Predicted'})
    ])
    
    fig = px.line(plot_df, x='Time', y='Power Output (kW)', color='Type', 
                  title=f'Wind Farm Power Output: {title}',
                  color_discrete_map={'Actual': '#2E86AB', 'Predicted': '#D64933'})
    
    fig.update_layout(
        title={'y': 0.95, 'x': 0.5, 'xanchor': 'center', 'yanchor': 'top', 'font': dict(size=20)},
        xaxis_title={'text': "Time of Measurement (10-minute intervals)", 'font': dict(size=14)},
        yaxis_title={'text': "Power Output (kW)", 'font': dict(size=14)},
        legend=dict(title="Values", bgcolor='rgba(255,255,255,0.8)', bordercolor='rgba(0,0,0,0.2)', borderwidth=1),
        plot_bgcolor='#f6f6f6', paper_bgcolor='#f6f6f6', width=1800, height=600, hovermode='x unified'
    )
    
    fig.update_traces(line=dict(width=2), selector=dict(name='Actual'))
    fig.update_traces(line=dict(width=2, dash='dash'), selector=dict(name='Predicted'))
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)', tickangle=45)
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')
    
    fig.show()

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_metrics(y_true: np.ndarray | pd.Series, y_pred: np.ndarray | pd.Series, dataset_name: str) -> dict:
    """
    Calculate and print regression metrics comparing predicted vs actual values.
    
    Args:
        y_true (np.ndarray | pd.Series): True/actual values
        y_pred (np.ndarray | pd.Series): Predicted values
        dataset_name (str): Name of the dataset for printing purposes
        
    Returns:
        dict: Dictionary containing the calculated metrics (rmse, mae, mape, r2)
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    print(f"\n{dataset_name} Metrics:")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE: {mae:.2f}") 
    print(f"MAPE: {mape:.2f}%")
    print(f"R2 Score: {r2:.3f}")
    
    return {
        "rmse": rmse,
        "mae": mae,
        "mape": mape,
        "r2": r2
    }

## Naive Baseline
linear regression ignoring the time series structure

In [10]:
# make pandas show 500 rows
pd.set_option('display.max_rows', 500)

# show rows that contain at least one NaN
len(df_train[df_train.isna().any(axis=1)]), len(df_val[df_val.isna().any(axis=1)])
# --> df_train and df_val still contain few rows with NaN values. Since we ignore the time series structure in this approach, we can just drop the rows with NaN values.

(276, 6)

In [11]:
df_train.info(show_counts=True, verbose=True)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30816 entries, 2020-05-01 00:00:00 to 2020-11-30 23:50:00
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   num_turbines       30816 non-null  int64  
 1   Day                30816 non-null  float64
 2   time               30816 non-null  object 
 3   Wspd               30553 non-null  float64
 4   Wdir               30545 non-null  float64
 5   Etmp               30558 non-null  float64
 6   Itmp               30558 non-null  float64
 7   Ndir               30558 non-null  float64
 8   maintenance_day    30816 non-null  float64
 9   turbine_stopped    30816 non-null  float64
 10  turbine_at_rest    30816 non-null  float64
 11  impute_day_patv    30816 non-null  float64
 12  Patv               30816 non-null  float64
 13  Patv_imputed       30816 non-null  float64
 14  n_active_turbines  30816 non-null  int64  
dtypes: float64(12), int64(2), object(1)

In [12]:
# remove rows with NaN values
df_train = df_train.dropna()
df_val = df_val.dropna()

df_train.info(show_counts=True, verbose=True)

# create X_train and y_train
X_train = df_train[features]
y_train = df_train[target]

# create X_val and y_val
X_val = df_val[features]
y_val = df_val[target]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 30540 entries, 2020-05-01 00:10:00 to 2020-11-30 23:50:00
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   num_turbines       30540 non-null  int64  
 1   Day                30540 non-null  float64
 2   time               30540 non-null  object 
 3   Wspd               30540 non-null  float64
 4   Wdir               30540 non-null  float64
 5   Etmp               30540 non-null  float64
 6   Itmp               30540 non-null  float64
 7   Ndir               30540 non-null  float64
 8   maintenance_day    30540 non-null  float64
 9   turbine_stopped    30540 non-null  float64
 10  turbine_at_rest    30540 non-null  float64
 11  impute_day_patv    30540 non-null  float64
 12  Patv               30540 non-null  float64
 13  Patv_imputed       30540 non-null  float64
 14  n_active_turbines  30540 non-null  int64  
dtypes: float64(12), int64(2), object(1)

In [13]:
import numpy as np
import statsmodels.api as sm
import mlflow

# Start MLflow run
with mlflow.start_run(run_name="linear_regression_naive_baseline"):
    print(f"Active run ID: {mlflow.active_run().info.run_id}")

    # Log parameters
    mlflow.log_param("model_type", "OLS")
    mlflow.log_param("features", list(X_train.columns))
    
    # remove Wdir from X_train and X_val
    X_train_sm = X_train.drop(columns=['Wdir'])
    X_val_sm = X_val.drop(columns=['Wdir'])
    mlflow.log_param("removed_features", ["Wdir"])

    # Add bias term
    X_train_sm = sm.add_constant(X_train_sm)
    X_val_sm = sm.add_constant(X_val_sm)

    # Create and train the model
    model = sm.OLS(y_train, X_train_sm)
    results = model.fit()

    # Make predictions
    train_pred = results.predict(X_train_sm)
    val_pred = results.predict(X_val_sm)

    # correct negative predictions
    train_pred = np.where(train_pred < 0, 0, train_pred)
    val_pred = np.where(val_pred < 0, 0, val_pred)

    # Calculate and log metrics
    train_metrics = calculate_metrics(y_train, train_pred, "Training")
    val_metrics = calculate_metrics(y_val, val_pred, "Validation")
    
    # Log metrics to MLflow
    mlflow.log_metrics({
        "train_rmse": train_metrics["rmse"],
        "train_mae": train_metrics["mae"],
        "train_mape": train_metrics["mape"],
        "train_r2": train_metrics["r2"],
        "val_rmse": val_metrics["rmse"],
        "val_mae": val_metrics["mae"], 
        "val_mape": val_metrics["mape"],
        "val_r2": val_metrics["r2"]
    })

    # Print model summary and coefficients
    print("\nModel Summary:")
    print(results.summary())
    
    # Log model coefficients as parameters
    for feature, coef in zip(X_train_sm.columns, results.params):
        mlflow.log_param(f"coef_{feature}", coef)

Active run ID: 16c87c0410844d7f8fe79bb7eb8c8dec

Training Metrics:
RMSE: 27.03
MAE: 17.24
MAPE: inf%
R2 Score: 0.748

Validation Metrics:
RMSE: 17.06
MAE: 11.76
MAPE: inf%
R2 Score: 0.858

Model Summary:
                            OLS Regression Results                            
Dep. Variable:           Patv_imputed   R-squared:                       0.733
Model:                            OLS   Adj. R-squared:                  0.733
Method:                 Least Squares   F-statistic:                 2.797e+04
Date:                Sun, 19 Jan 2025   Prob (F-statistic):               0.00
Time:                        18:11:46   Log-Likelihood:            -1.4490e+05
No. Observations:               30540   AIC:                         2.898e+05
Df Residuals:                   30536   BIC:                         2.898e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
      

The feature 'Wdir' has a p value of 0.33, indicating that its influence on the target variable may not be statistically significant in the lin reg. Thus, remove it and evaluate again

In [ ]:
# Plot training and validation predictions
plot_predictions(y_train, train_pred, 'Training Set Predictions vs Actual Values')
plot_predictions(y_val, val_pred, 'Validation Set Predictions vs Actual Values')

In [ ]:
# todo invetigate: pred is off hard (too high). probably some issue with the number of active turbines that I calculated

## Simple GBT regression approach

Hypothesis: the power output mainly depends on the external weather conditions at that time: wind speed, wind direction, temperature (+ the number of active turbines). The historical output prior to t+1 is not that informative for t+1  (as indicated by the autocorrelation analysis). In this case, a simple linear regression model or LightGBM without any time series features should perform well.

In [ ]:
# TODO: CONTINUE HERE implement lightgbm and xgboost here

## Multi-Output Model Approach

Predict all steps at once using MultiOutputRegressor (scikit-learn)

"create one instance of the provided model for each output" --> Would be similar to Muilti-step direct forecast but one model instantiated many times with single hyper param tuning

In [ ]:
# TODO: implement the approach from this paper https://arxiv.org/pdf/2101.02118. Is appearently also used here https://towardsdatascience.com/multi-step-time-series-forecasting-with-xgboost-65d6820bec39 
# their approach seems to be to create a feature for each time step in the input window of the target variable (e.g., use y_t-1, y_t-2, ... as features for y_t). Plus, they use the last value of each of the covariate time series as a feature.

# if this yields significantly better results than in the simple regression approach, the past values of the power output indeed contain useful (autoregressive) information to predict the future. The autocorrelation analyses suggested otherwise, but ofc this just regarded linear correlation.

## Multi-Step Direct Forecast

Separate Model per Horizon.

In the challenge, the task is to predict the next 2 days at 10 minute intervals meaning 6 measurements per hour * 24 hours / day * 2 days = 288 steps. 

Advantage: No error accumulation over time.

Disadvantage: assumes the sequential power generation values are independent. Training and tuning hyper parameters for 288 models is not efficient, so we skip this approach.



## Recursive Multi-Step
Use predictions from previous steps as features for the next step (errors accumulate over time though)

## Seq2Seq Model
Use NN architecture that handles sequential data naturally (e.g., LSTM or GRU) to directly model the temporal dependencies